In [1]:
import numpy as np
import pandas as pd

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float,
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str,
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int,
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str,
              'sqft_lot':int, 'view':int,
             }

In [3]:
df_sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [4]:
def get_data(data, features, target):
    out_df = pd.DataFrame(data[features])
    out_df['constant'] = 1
    
    return out_df[['constant'] + features], pd.DataFrame(data[target])

In [5]:
def predict_output(feature_matrix, weights):
    return np.dot(feature_matrix, weights)

In [6]:
def normalise(features):
    norms = np.sqrt(np.diag(np.dot(features.T, features)))
    normalised_features = np.divide(features, norms)
    
    return normalised_features, norms.reshape(-1,1)

In [7]:
def normalise2(features):
    
    norms = np.linalg.norm(features, axis=0)
    normalised_features = features / norms
    
    return normalised_features, norms.reshape(-1,1)

Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.

- First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
- Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
- Set initial weights to [1,4,1].
- Make predictions with feature matrix and initial weights.
- Compute values of ro[i], where
<pre>ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]</pre>

In [8]:
feature_matrix, target_matrix = get_data(df_sales, ['sqft_living', 'bedrooms'], 'price')
normalised_features, norms = normalise(feature_matrix)
initial_weights = np.array([1,4,1]).reshape(-1,1)
predictions = predict_output(normalised_features, initial_weights)

In [9]:
def get_rho(features, target, predictions, weights):    
    return np.sum(features*(target.values - predictions + weights.T * features), axis=0)

In [10]:
def get_rho2(i, features, target, prediction, weights):
    return np.sum(features.values[:,i].reshape(-1,1) * ((target - prediction).values + (weights[i][0] * features.values[:,i]).reshape(-1,1))).reshape(-1,1)

In [11]:
get_rho(normalised_features, target_matrix, predictions, initial_weights)

constant       7.940030e+07
sqft_living    8.793947e+07
bedrooms       8.096670e+07
dtype: float64

<hr>
**Quiz Question: Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?**
<hr>

w[2] = 80966698.66623947

In [12]:
80966698.66623947 * 2

161933397.33247894

<hr>
** Quiz Question: What range of values of l1_penalty would set both w[1] and w[2] to zero, if we were to take a step in that coordinate? **

w[1] = 87939470.82325175

In [13]:
87939470.82325175 * 2

175878941.6465035

<hr>

In [14]:
def lasso_coordinate_descent_step(i, feature_matrix, target, weights, l1_penalty):
    prediction =  predict_output(feature_matrix, weights)

    rho_i = get_rho2(i, feature_matrix, target, prediction, weights)
    
    if i == 0:
        new_weight_i = rho_i
    elif rho_i < -l1_penalty/2.:
        new_weight_i = rho_i + l1_penalty/2.
    elif rho_i > l1_penalty/2.:
        new_weight_i = rho_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

testing function

In [15]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1,
                                    pd.DataFrame(np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]])),
                                    pd.DataFrame(np.array([1., 1.]).reshape(-1,1)),
                                    np.array([1., 4.]).reshape(-1,1),
                                    0.1,
                                   )
     )


[[0.42555885]]


In [16]:
def lasso_cyclical_coordinate_descent(feature_matrix, target, initial_weights, l1_penalty, tolerance):
    
    changes = np.full(len(feature_matrix.columns),tolerance+1)
    weights = initial_weights
    
    while changes.max() >= tolerance:
    
        for i in range(len(feature_matrix.columns)):
            new_weight = lasso_coordinate_descent_step(i=i, 
                                          feature_matrix=feature_matrix,
                                          target=target,
                                          weights=weights,
                                          l1_penalty=l1_penalty
                                         )

            changes[i] = abs(weights[i] - new_weight)
            weights[i] = new_weight
    
    
    return weights

In [17]:
feature_matrix, target_matrix = get_data(df_sales, ['sqft_living', 'bedrooms'], 'price')
normalised, norms = normalise(feature_matrix)
initial_weights = np.zeros(len(normalised.columns)).reshape(-1,1)
l1_penalty = 1e7
tolerance = 1.0

In [18]:
weights = lasso_cyclical_coordinate_descent(feature_matrix=normalised,
                                  target=target_matrix,
                                  initial_weights=initial_weights,
                                  l1_penalty=l1_penalty,
                                  tolerance=tolerance,
                                 )
weights

array([[21624997.9595191 ],
       [63157247.20788956],
       [       0.        ]])

In [19]:
rss = lambda y, y_hat: np.dot((y - y_hat).T, (y - y_hat))

In [20]:
rss(target_matrix, predict_output(normalised, weights))

array([[1.63049248e+15]])

<hr>
**Quiz Question: What is the RSS of the learned model on the normalized dataset?**

**Quiz Question: Which features had weight zero at convergence?**
<hr>

In [21]:
features = [
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "sqft_lot",
    "floors",
    "waterfront",
    "view",
    "condition",
    "grade",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
]

In [22]:
df_train = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
df_test = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

In [23]:
features_train, target_train = get_data(data=df_train, features=features, target='price')
normalised_train, norms_train = normalise2(features_train)

In [24]:
weights_1e7 = lasso_cyclical_coordinate_descent(feature_matrix=normalised_train,
                                                target=target_train,
                                                initial_weights=np.zeros(len(normalised_train.columns)).reshape(-1,1),
                                                l1_penalty=1e7,
                                                tolerance=1.0,
                                 )

<hr>
**Quiz Question: What features had non-zero weight in this case?**

In [25]:
dict(zip(['constant'] + features, weights_1e7[:,0]))

{'constant': 24429600.234403126,
 'bedrooms': 0.0,
 'bathrooms': 0.0,
 'sqft_living': 48389174.77154895,
 'sqft_lot': 0.0,
 'floors': 0.0,
 'waterfront': 3317511.214921655,
 'view': 7329961.811714256,
 'condition': 0.0,
 'grade': 0.0,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'yr_built': 0.0,
 'yr_renovated': 0.0}

<hr>
Next, learn the weights with l1_penalty=1e8

**Quiz Question: What features had non-zero weight in this case?**

In [26]:
weights_1e8 = lasso_cyclical_coordinate_descent(feature_matrix=normalised_train,
                                                target=target_train,
                                                initial_weights=np.zeros(len(normalised_train.columns)).reshape(-1,1),
                                                l1_penalty=1e8,
                                                tolerance=1.0,
                                 )
dict(zip(['constant'] + features, weights_1e8[:,0]))

{'constant': 71114625.71488702,
 'bedrooms': 0.0,
 'bathrooms': 0.0,
 'sqft_living': 0.0,
 'sqft_lot': 0.0,
 'floors': 0.0,
 'waterfront': 0.0,
 'view': 0.0,
 'condition': 0.0,
 'grade': 0.0,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'yr_built': 0.0,
 'yr_renovated': 0.0}

<hr>
Finally, learn the weights with l1_penalty=1e4

**Quiz Question: What features had non-zero weight in this case?**

In [27]:
weights_1e4 = lasso_cyclical_coordinate_descent(feature_matrix=normalised_train,
                                                target=target_train,
                                                initial_weights=np.zeros(len(normalised_train.columns)).reshape(-1,1),
                                                l1_penalty=1e4,
                                                tolerance=1.0,
                                 )
dict(zip(['constant'] + features, weights_1e4[:,0]))

{'constant': 787859491.4777097,
 'bedrooms': -19661143.86195114,
 'bathrooms': 12889336.680140473,
 'sqft_living': 52001512.37648063,
 'sqft_lot': -1344483.463465898,
 'floors': 4406130.558392141,
 'waterfront': 6858434.733728389,
 'view': 4535871.1556754075,
 'condition': 8664230.48212697,
 'grade': 128374341.98013332,
 'sqft_above': 0.0,
 'sqft_basement': 364776.37068867375,
 'yr_built': -898872854.9727703,
 'yr_renovated': 649411.4723077354}

In [28]:
normalised_weights_1e7 = weights_1e7 / norms_train
normalised_weights_1e7[3]

array([161.31745765])

Evaluating each of the learned models on the test data

In [29]:
features_test, target_test = get_data(df_test, features, 'price')

In [30]:
normalised_weights_1e8 = weights_1e8 / norms_train
normalised_weights_1e4 = weights_1e4 / norms_train

In [31]:
rss_1e7 = rss(target_test, predict_output(features_test, normalised_weights_1e7))
rss_1e8 = rss(target_test, predict_output(features_test, normalised_weights_1e8))
rss_1e4 = rss(target_test, predict_output(features_test, normalised_weights_1e4))

print('''
rss_1e7: {}
rss_1e8: {}
rss_1e4: {}
'''.format(rss_1e7, rss_1e8, rss_1e4))


rss_1e7: [[2.75962076e+14]]
rss_1e8: [[5.37166151e+14]]
rss_1e4: [[1.94415789e+14]]

